# Labeling

In [ ]:
test = pd.read_csv('TestActivities2.csv')

In [ ]:
test

,ID,Started,Finished,Updated,Subject,count
0,1156840,2024-09-08 22:47:00,2024-09-08 22:47:00,2024-09-08 22:47:00,U11,6
1,1156841,2024-09-08 22:47:00,2024-09-08 22:47:00,2024-09-08 22:47:00,U11,6
2,1156849,2024-09-08 22:47:00,2024-09-08 22:47:00,2024-09-08 22:47:00,U11,6
3,1156850,2024-09-08 22:47:00,2024-09-08 22:47:00,2024-09-08 22:47:00,U11,6
4,1156851,2024-09-08 22:47:00,2024-09-08 22:47:00,2024-09-08 22:47:00,U11,6
...,...,...,...,...,...,...
159,1171540,2024-09-12 19:36:00,2024-09-12 19:36:00,2024-09-12 19:36:00,U12,2
160,1171541,2024-09-12 19:36:00,2024-09-12 19:37:00,2024-09-12 19:37:00,U12,2
161,1171542,2024-09-12 19:37:00,2024-09-12 19:37:00,2024-09-12 19:37:00,U12,2
162,1171543,2024-09-12 19:37:00,2024-09-12 19:38:00,2024-09-12 19:38:00,U12,2


In [ ]:
len(os.listdir('output_csvs_test/'))

164

In [ ]:
result = os.listdir('result')

In [ ]:
def extract_times(filename):
    """Extracts the start and end times from a filename.

    Args:
        filename: The filename string.

    Returns:
        A tuple containing the start and end times as strings, or None if the
        filename doesn't match the expected pattern.
    """
    match = re.search(r"_(\d{4}-\d{2}-\d{2} \d{2}_\d{2}_\d{2})_(\d{4}-\d{2}-\d{2} \d{2}_\d{2}_\d{2})_", filename)
    if match:
        start_time = match.group(1)
        end_time = match.group(2)
        return start_time, end_time
    return None

In [ ]:
from collections import defaultdict
import re

In [ ]:
# 1. Create the directory if it doesn't exist
output_folder = "start_time"
os.makedirs(output_folder, exist_ok=True)  # exist_ok=True prevents error if folder exists


In [ ]:
# 1. Specify the input folder
input_folder = "result"


# 2. Get the list of CSV files from the input folder
file_list = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith(".csv")]

# 4. Group and merge files (same logic as before)
files_by_start_time = defaultdict(list)
for file in file_list:
    start_time, _ = extract_times(file)
    if start_time:  # Check if extract_times was successful
        files_by_start_time[start_time].append(file)
    else:
        print(f"Warning: Could not extract time from {file}. Skipping.")


for start_time, files in files_by_start_time.items():
    dfs = []
    for file in files:
        try:
            df = pd.read_csv(file)
            dfs.append(df)
        except FileNotFoundError:
            print(f"Warning: File not found: {file}. Skipping.")
        except pd.errors.EmptyDataError:
            print(f"Warning: File is empty: {file}. Skipping.")
        except pd.errors.ParserError:
            print(f"Warning: Error parsing file: {file}. Skipping.")
        except Exception as e: # Catching other potential errors during reading
            print(f"Warning: An error occurred while reading file: {file}. Skipping. Error details: {e}")

    if dfs: # Check if there are any dataframes to merge
        merged_df = pd.concat(dfs, ignore_index=True)
        merged_filename = os.path.join(output_folder, f"{start_time}.csv")
        merged_df.to_csv(merged_filename, index=False)
        print(f"Merged files with start time {start_time} into {merged_filename}")
    else:
        print(f"No valid CSV files found for start time {start_time}. Skipping.")

In [ ]:
pd.read_csv("start_time/" + os.listdir('start_time/')[0])

,Timestamp,X,Y,Z,predict
0,2024-09-08 22:48:03.057,0.407,5.071,8.406,3
1,2024-09-08 22:48:03.158,0.211,5.114,8.210,3
2,2024-09-08 22:48:03.177,0.191,5.157,8.083,3
3,2024-09-08 22:48:03.249,0.345,5.097,8.521,3
4,2024-09-08 22:48:03.256,0.366,5.145,8.200,3
...,...,...,...,...,...
153,2024-09-08 22:48:08.790,3.642,2.193,9.972,3
154,2024-09-08 22:48:08.885,1.288,0.656,17.212,3
155,2024-09-08 22:48:08.903,3.261,4.753,-1.994,3
156,2024-09-08 22:48:08.977,5.854,3.790,4.489,3


In [ ]:
os.listdir('start_time/')[0]

'2024-09-08 22_48_00.csv'

In [ ]:
len(os.listdir('start_time/'))

164

In [ ]:
def create_dataframe_with_max_counts_and_mode(folder_path):
    """
    Creates a Pandas DataFrame with filenames, maximum counts, and the most frequent value
    in the 'predict' column of each file.

    Args:
        folder_path (str): The path to the folder containing the CSV files.

    Returns:
        pandas.DataFrame: A DataFrame with three columns: 'file_name', 'max_count', and 'predict'.
                           Returns an empty DataFrame if no CSV files are found or if an error occurs.
    """

    results = []
    try:
        for filename in os.listdir(folder_path):
            if filename.endswith(".csv"):
                file_path = os.path.join(folder_path, filename)
                try:
                    df = pd.read_csv(file_path)
                    if 'predict' in df.columns:
                        value_counts = df['predict'].value_counts()
                        max_count = value_counts.max()
                        mode = value_counts.idxmax()  # Get the most frequent value (mode)

                        results.append({'file_name': filename, 'max_count': max_count, 'predict': mode})
                    else:
                        print(f"Warning: 'predict' column not found in {filename}. Skipping.")
                except pd.errors.EmptyDataError:
                    print(f"Warning: File {filename} is empty. Skipping.")
                except pd.errors.ParserError:
                    print(f"Warning: Error parsing file {filename}. Skipping.")
                except Exception as e:
                    print(f"Warning: An error occurred while processing {filename}: {e}. Skipping.")


    except FileNotFoundError:
        print(f"Error: Folder '{folder_path}' not found.")
        return pd.DataFrame(results)

    df_results = pd.DataFrame(results)
    return df_results


# Example usage:
folder_path = "start_time/"  # Replace with the actual path to your folder
df = create_dataframe_with_max_counts_and_mode(folder_path)

if not df.empty:
    print(df)
else:
    print("No valid CSV files found or an error occurred.")

                   file_name  max_count  predict
0    2024-09-08 22_48_00.csv        158        3
1    2024-09-08 22_47_40.csv        105        3
2    2024-09-08 22_47_30.csv        105        4
3    2024-09-08 22_47_50.csv        157        3
4    2024-09-08 22_47_20.csv        105        3
..                       ...        ...      ...
159  2024-09-12 19_37_00.csv        632        1
160  2024-09-12 19_38_00.csv       1736       10
161  2024-09-12 19_37_30.csv        467        9
162  2024-09-12 19_36_30.csv        782        9
163  2024-09-12 19_36_00.csv        790        9

[164 rows x 3 columns]


In [ ]:
test

,ID,Started,Finished,Updated,Subject,count
0,1156840,2024-09-08 22:47:00,2024-09-08 22:47:00,2024-09-08 22:47:00,U11,6
1,1156841,2024-09-08 22:47:00,2024-09-08 22:47:00,2024-09-08 22:47:00,U11,6
2,1156849,2024-09-08 22:47:00,2024-09-08 22:47:00,2024-09-08 22:47:00,U11,6
3,1156850,2024-09-08 22:47:00,2024-09-08 22:47:00,2024-09-08 22:47:00,U11,6
4,1156851,2024-09-08 22:47:00,2024-09-08 22:47:00,2024-09-08 22:47:00,U11,6
...,...,...,...,...,...,...
159,1171540,2024-09-12 19:36:00,2024-09-12 19:36:00,2024-09-12 19:36:00,U12,2
160,1171541,2024-09-12 19:36:00,2024-09-12 19:37:00,2024-09-12 19:37:00,U12,2
161,1171542,2024-09-12 19:37:00,2024-09-12 19:37:00,2024-09-12 19:37:00,U12,2
162,1171543,2024-09-12 19:37:00,2024-09-12 19:38:00,2024-09-12 19:38:00,U12,2


In [ ]:
test['predict'] = df['predict']

In [ ]:
test.to_csv("TestActivities_predict.csv", index = False)

In [ ]:
# prompt: remove the column 'count' of test

test = test.drop('count', axis=1)


In [ ]:
test

,ID,Started,Finished,Updated,Subject,predict
0,1156840,2024-09-08 22:47:00,2024-09-08 22:47:00,2024-09-08 22:47:00,U11,3
1,1156841,2024-09-08 22:47:00,2024-09-08 22:47:00,2024-09-08 22:47:00,U11,3
2,1156849,2024-09-08 22:47:00,2024-09-08 22:47:00,2024-09-08 22:47:00,U11,4
3,1156850,2024-09-08 22:47:00,2024-09-08 22:47:00,2024-09-08 22:47:00,U11,3
4,1156851,2024-09-08 22:47:00,2024-09-08 22:47:00,2024-09-08 22:47:00,U11,3
...,...,...,...,...,...,...
159,1171540,2024-09-12 19:36:00,2024-09-12 19:36:00,2024-09-12 19:36:00,U12,1
160,1171541,2024-09-12 19:36:00,2024-09-12 19:37:00,2024-09-12 19:37:00,U12,10
161,1171542,2024-09-12 19:37:00,2024-09-12 19:37:00,2024-09-12 19:37:00,U12,9
162,1171543,2024-09-12 19:37:00,2024-09-12 19:38:00,2024-09-12 19:38:00,U12,9


In [ ]:
test['predict'].value_counts()

,count
predict,
3,48
1,29
9,22
2,22
4,16
5,9
6,8
10,5
8,4


In [ ]:
test.to_csv("TestActivities_predict2.csv", index = False)